---

# Audiobook Generator - Proof of Concept Notebook

This notebook is intended to be a proof of concept for the end-to-end work of generating an audiobook file from an ebook. This includes converting the .epub book files into raw python text strings, splitting into items and sentences, then tokenizing and batching them to run through the Silero text-to-speech (TTS) implementation.

*Updated: September 2, 2022*

---

### Overview

1. Setup
    - Needed libraries and packages
    - Variables
    - Silero model selection
2. Ebook Import
    - Target file selection
    - File (.epub) import
    - String parsing
    - String length wrapping
3. Text-to-Speech
    - Silero implementation
    - Results

---

### Step 1 - Setup

This proof-of-concept relies on PyTorch and TorchAudio for its implementation. OmegaConf is used to support providing the latest model from Silero in a consistent manner. A seed is created, and used for all random function that are needed.

We will also use the TQDM package to provide progress bars while running the proof-of-concept within this notebook.

In [1]:
import os
import torch
import torchaudio
from omegaconf import OmegaConf
from tqdm.notebook import tqdm

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

seed = 1337
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

We also need to set some variables for later use during the text processing steps, and the audio output in the TTS step.

- `max_char_len` is set based on the results of performance testing done by the Silero devs. Larger values enable sentence structure to be better preserved, but negatively affect performance.
- `sample_rate` is also set based on recommendations from the Silero team for performance vs. quality. Using 16k or 8k audio will improve performance, but result in lower quality audio. Silero estimates a decrease of ~0.5 MOS (from 3.7 to 3.2).

In [2]:
max_char_len = 150
sample_rate = 24000

The Silero implementation comes with models trained on various languages, the most common being Russian, but we will use the latest English model for this proof of concept. There are also a number of English speaker choices available.

In [3]:
language = 'en'
model_id = 'v3_en'
speaker = 'en_0'

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

Using cache found in /home/mkutarna/.cache/torch/hub/snakers4_silero-models_master


### Step 2 - Ebook Import

Below is a representative ebook (`Protrait of Dorian Gray`), taken from Project Gutenberg - a free directory of public-domain works.

In [4]:
txt_path = 'test.txt'

The function below - `read_ebook()` - performs the following steps:
- Takes in the ebook, located at `ebook_path`
- Strips out any html tags
- Uses the nltk packages to download and use the `punkt` sentence-level tokenizer
- Calls the TextWrapper package to wrap sentences to the `max_char_len`, with care to fix sentence endings
- I.e. sentences are not split in the middle of a word, but rather words are preserved
- Finally sentences are appended to a chapter, and the chapters to a complete list: `corpus`

In [5]:
import re
import numpy as np
from pathlib import Path
from bs4 import BeautifulSoup
from nltk import tokenize, download
from textwrap import TextWrapper

def read_txt(txt_path):      
    download('punkt', quiet=True)
#     wrapper = TextWrapper(cf.MAX_CHAR_LEN, fix_sentence_endings=True)
    wrapper = TextWrapper(140, fix_sentence_endings=True)

    with open('test.txt') as f:
        book = f.read()
    
    input_text = BeautifulSoup(book, "html.parser").text
    text_list = []
    for paragraph in input_text.split('\n'):
        paragraph = paragraph.replace('—', '-')
#         paragraph = re.sub(r'[^\x00-\x7f]', "", paragraph)
        sentences = tokenize.sent_tokenize(paragraph)
        
        # Wrap sentences to maximum character limit
        sentence_list = []
        for sentence in sentences:
            wrapped_sentences = wrapper.wrap(sentence)
            sentence_list.append(wrapped_sentences)
            
        # Flatten list of list of sentences and append
        trunc_sentences = [phrase for sublist in sentence_list for phrase in sublist]
        text_list.append(trunc_sentences)
    text_list = [[text for sentences in text_list for text in sentences]]

    return text_list

Here we use the above function to read in the chosen ebook.

In [6]:
corpus = read_txt(txt_path)
title = 'character_testing'

And here, let us take a peak at the contents of the ebook:

In [7]:
print(f'Title of ebook (path name):{title}')
print(f'First paragraph (truncated for display): \n {corpus[0]}')

Title of ebook (path name):character_testing
First paragraph (truncated for display): 
 ['Testing Text File', 'Built to test various characters and other possible inputs to the silero model.', 'Here are some Chinese characters: 此行是对非英语字符的测试.', 'The Greek alphabet was developed to write the Greek language about 1000 BC.', 'It is the ancestor of modern languages and is derived from the Phoenician alphabet.', 'There are 24 letters in the Greek alphabet.', 'The vowels: are α, ε, η, ι, ο, ω, υ.', 'All the rest are consonants.', 'Alpha is the first letter of the alphabet, and the last letter is omega.', 'In addition, writing the Greek language, the letters are also widely used as symbols in mathematics, chemistry, engineering, physics, and', 'science today.', 'We can also test from mathematical symbols: ∫, ∇, ∞, δ, ε, X̄, %, √ ,a, ±, ÷, +, = ,-.', 'Finally, here are some emoticons: ☺️🙂😊😀😁☹️🙁😞😟😣😖😨😧😦😱😫😩.']


### Step 3 - Text-to-Speech

The ebook is fed through the Silero TTS implementation sentence by sentence. We will also check that each tensor being created is valid (i.e. non-zero).

Finally, the output tensors are exported as `.wav` files on a chapter by chapter basis - consistent with the file structure of common audiobooks.

In [8]:
len(corpus[0])

13

In [9]:
#os.mkdir(f'outputs/{title}')

for chapter in tqdm(corpus):
    chapter_index = f'chapter{corpus.index(chapter):03}'
    audio_list = []
    for sentence in tqdm(chapter):
        print(f'Current sentence: {sentence}')
        audio = model.apply_tts(text=sentence,
                                speaker=speaker,
                                sample_rate=sample_rate)
        if len(audio) > 0 and isinstance(audio, torch.Tensor):
            audio_list.append(audio)
        else:
            print(f'Tensor for sentence is not valid: \n {sentence}')

    sample_path = f'outputs/{title}_{chapter_index}.wav'

    if len(audio_list) > 0:
        audio_file = torch.cat(audio_list).reshape(1, -1)
        torchaudio.save(sample_path, audio_file, sample_rate)
    else:
        print(f'Chapter {chapter_index} is empty.')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Current sentence: Testing Text File
Current sentence: Built to test various characters and other possible inputs to the silero model.
Current sentence: Here are some Chinese characters: 此行是对非英语字符的测试.
Current sentence: The Greek alphabet was developed to write the Greek language about 1000 BC.
Current sentence: It is the ancestor of modern languages and is derived from the Phoenician alphabet.
Current sentence: There are 24 letters in the Greek alphabet.
Current sentence: The vowels: are α, ε, η, ι, ο, ω, υ.
Current sentence: All the rest are consonants.
Current sentence: Alpha is the first letter of the alphabet, and the last letter is omega.
Current sentence: In addition, writing the Greek language, the letters are also widely used as symbols in mathematics, chemistry, engineering, physics, and
Current sentence: science today.
Current sentence: We can also test from mathematical symbols: ∫, ∇, ∞, δ, ε, X̄, %, √ ,a, ±, ÷, +, = ,-.
Current sentence: Finally, here are some emoticons: ☺️🙂

### Results

##### CPU (i7-4790k)

Running "Pride and Prejudice" through the Silero model took **34m42s** to convert. This book is a good representation of the average book length: the average audiobook length on Audible is between 10 & 12 hours, while Pride and Prejudice is 11h20m.

This is approximately a 20:1 ratio of audio length to processing time.

Pride and Prejudice: **34m42s** - 1h39m33s on i7-4650u

Portrait of Dorian Gray: **18m18s** - 18m50s w/output - 1h06hm04s on i7-4650u

Crime and Punishment: **Unknown** - error converting ebook at 7/50, 19/368

##### GPU (P4000)

Running the same book through the Silero model on GPU took **5m39s** to convert.

This is approximately a 122:1 ratio of audio length to processing time.

Pride and Prejudice: **5m39s**

Portrait of Dorian Gray: **4m26s**

Crime and Punishment: **Unknown** - error converting ebook